【Point】

- INITIAL_BITSの値とマイニングにかかる時間を確認しましょう！

In [1]:
import hashlib
import datetime
import time
import json

INITIAL_BITS = 0x1e777777
MAX_32BIT = 0xffffffff

class Block():
    def __init__(self, index, prev_hash, data, timestamp, bits):
        self.index = index
        self.prev_hash = prev_hash
        self.data = data
        self.timestamp = timestamp
        self.bits = bits
        self.nonce = 0
        self.elapsed_time = ""
        self.block_hash = ""

    def __setitem__(self, key, value):
        setattr(self, key, value)

    def to_json(self):
        return {
            "index"       : self.index,
            "prev_hash"   : self.prev_hash,
            "stored_data" : self.data,
            "timestamp"   : self.timestamp.strftime("%Y/%m/%d %H:%M:%S"),
            "bits"        : hex(self.bits)[2:].rjust(8, "0"),
            "nonce"       : hex(self.nonce)[2:].rjust(8, "0"),
            "elapsed_time": self.elapsed_time,
            "block_hash"  : self.block_hash
        }

    def calc_blockhash(self):
        blockheader = str(self.index) + str(self.prev_hash) + str(self.data) + str(self.timestamp) + hex(self.bits)[2:] + str(self.nonce)
        h = hashlib.sha256(blockheader.encode()).hexdigest()
        self.block_hash = h
        return h
    
    def calc_target(self):
        exponent_bytes = (self.bits >> 24) - 3
        exponent_bits = exponent_bytes * 8
        coefficient = self.bits & 0xffffff
        return coefficient << exponent_bits
    
    def check_valid_hash(self):
        return int(self.calc_blockhash(), 16) <= self.calc_target()

class Blockchain():
    def __init__(self, initial_bits):
        self.chain = []
        self.initial_bits = initial_bits

    def add_block(self, block):
        self.chain.append(block)
    
    def getblockinfo(self, index=-1):
        return print(json.dumps(self.chain[index].to_json(), indent=2, sort_keys=True, ensure_ascii=False))
    
    def mining(self, block):
        start_time = int(time.time() * 1000)
        while True:
            for n in range(MAX_32BIT + 1):
                block.nonce = n
                if block.check_valid_hash():
                    end_time = int(time.time() * 1000)
                    block.elapsed_time = str((end_time - start_time) / 1000.0) + "秒"
                    self.add_block(block)
                    self.getblockinfo()
                    return
            new_time = datetime.datetime.now()
            if new_time == block.timestamp:
                block.timestamp += datetime.timedelta(seconds=1)
            else:
                block.timestamp = new_time
    
    def create_genesis(self):
        genesis_block = Block(0, "0000000000000000000000000000000000000000000000000000000000000000", "ジェネシスブロック", datetime.datetime.now(), self.initial_bits)
        self.mining(genesis_block)
    
    def add_newblock(self, i):
        last_block = self.chain[-1]
        block = Block(i+1, last_block.block_hash, "ブロック " + str(i+1), datetime.datetime.now(), last_block.bits)
        self.mining(block)

if __name__ == "__main__":
    bc = Blockchain(INITIAL_BITS)
    print("ジェネシスブロックを作成中・・・")
    bc.create_genesis()
    for i in range(30):
        print(str(i+2) + "番目のブロックを作成中・・・")
        bc.add_newblock(i)

ジェネシスブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "00001784f96090c36dc89e0a55f833db4f7b794fa92c6ec21ded61854d94077b",
  "elapsed_time": "0.037秒",
  "index": 0,
  "nonce": "000022c2",
  "prev_hash": "0000000000000000000000000000000000000000000000000000000000000000",
  "stored_data": "ジェネシスブロック",
  "timestamp": "2019/10/29 13:19:15"
}
2番目のブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "000060356f24a31d428551ec4d2b9b8536d7c4c093eecece9b136dc5a435d7ff",
  "elapsed_time": "0.05秒",
  "index": 1,
  "nonce": "000030d4",
  "prev_hash": "00001784f96090c36dc89e0a55f833db4f7b794fa92c6ec21ded61854d94077b",
  "stored_data": "ブロック 1",
  "timestamp": "2019/10/29 13:19:15"
}
3番目のブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "00003db15281e8c687c4d303144abbe1fdb98e35882ee2fabb9618b887ebc667",
  "elapsed_time": "0.084秒",
  "index": 2,
  "nonce": "000056b8",
  "prev_hash": "000060356f24a31d428551ec4d2b9b8536d7c4c093eecece9b136dc5a435d7ff",
  "stored_data": "ブロック 2",
  "timestamp": "2019/1

{
  "bits": "1e777777",
  "block_hash": "000069ef013919979d84c40326b8c3fa0e0c4c682bca37524247c1eacd8359b5",
  "elapsed_time": "1.001秒",
  "index": 25,
  "nonce": "0003e241",
  "prev_hash": "000075575889b7a96f4c42a5d017b46ccdd7f4ccff4594677961a879a258bf55",
  "stored_data": "ブロック 25",
  "timestamp": "2019/10/29 13:19:32"
}
27番目のブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "00004ff49b289a790edca53ff62c0c1f96bd7a590dbdb61c9673b31d1ae9f7f7",
  "elapsed_time": "0.944秒",
  "index": 26,
  "nonce": "00039eb9",
  "prev_hash": "000069ef013919979d84c40326b8c3fa0e0c4c682bca37524247c1eacd8359b5",
  "stored_data": "ブロック 26",
  "timestamp": "2019/10/29 13:19:33"
}
28番目のブロックを作成中・・・
{
  "bits": "1e777777",
  "block_hash": "00006f12a8adc8b36acfc8a700f725ffa9402eb4080983bce60318f466fbbeee",
  "elapsed_time": "0.95秒",
  "index": 27,
  "nonce": "0003a11b",
  "prev_hash": "00004ff49b289a790edca53ff62c0c1f96bd7a590dbdb61c9673b31d1ae9f7f7",
  "stored_data": "ブロック 27",
  "timestamp": "2019/10/29 13:19:3

In [2]:
import hashlib
import datetime
import time
import json

INITIAL_BITS = 0x1d777777
MAX_32BIT = 0xffffffff

class Block():
    def __init__(self, index, prev_hash, data, timestamp, bits):
        self.index = index
        self.prev_hash = prev_hash
        self.data = data
        self.timestamp = timestamp
        self.bits = bits
        self.nonce = 0
        self.elapsed_time = ""
        self.block_hash = ""

    def __setitem__(self, key, value):
        setattr(self, key, value)

    def to_json(self):
        return {
            "index"       : self.index,
            "prev_hash"   : self.prev_hash,
            "stored_data" : self.data,
            "timestamp"   : self.timestamp.strftime("%Y/%m/%d %H:%M:%S"),
            "bits"        : hex(self.bits)[2:].rjust(8, "0"),
            "nonce"       : hex(self.nonce)[2:].rjust(8, "0"),
            "elapsed_time": self.elapsed_time,
            "block_hash"  : self.block_hash
        }

    def calc_blockhash(self):
        blockheader = str(self.index) + str(self.prev_hash) + str(self.data) + str(self.timestamp) + hex(self.bits)[2:] + str(self.nonce)
        h = hashlib.sha256(blockheader.encode()).hexdigest()
        self.block_hash = h
        return h
    
    def calc_target(self):
        exponent_bytes = (self.bits >> 24) - 3
        exponent_bits = exponent_bytes * 8
        coefficient = self.bits & 0xffffff
        return coefficient << exponent_bits
    
    def check_valid_hash(self):
        return int(self.calc_blockhash(), 16) <= self.calc_target()

class Blockchain():
    def __init__(self, initial_bits):
        self.chain = []
        self.initial_bits = initial_bits

    def add_block(self, block):
        self.chain.append(block)
    
    def getblockinfo(self, index=-1):
        return print(json.dumps(self.chain[index].to_json(), indent=2, sort_keys=True, ensure_ascii=False))
    
    def mining(self, block):
        start_time = int(time.time() * 1000)
        while True:
            for n in range(MAX_32BIT + 1):
                block.nonce = n
                if block.check_valid_hash():
                    end_time = int(time.time() * 1000)
                    block.elapsed_time = str((end_time - start_time) / 1000.0) + "秒"
                    self.add_block(block)
                    self.getblockinfo()
                    return
            new_time = datetime.datetime.now()
            if new_time == block.timestamp:
                block.timestamp += datetime.timedelta(seconds=1)
            else:
                block.timestamp = new_time
    
    def create_genesis(self):
        genesis_block = Block(0, "0000000000000000000000000000000000000000000000000000000000000000", "ジェネシスブロック", datetime.datetime.now(), self.initial_bits)
        self.mining(genesis_block)
    
    def add_newblock(self, i):
        last_block = self.chain[-1]
        block = Block(i+1, last_block.block_hash, "ブロック " + str(i+1), datetime.datetime.now(), last_block.bits)
        self.mining(block)

if __name__ == "__main__":
    bc = Blockchain(INITIAL_BITS)
    print("ジェネシスブロックを作成中・・・")
    bc.create_genesis()
    for i in range(30):
        print(str(i+2) + "番目のブロックを作成中・・・")
        bc.add_newblock(i)

ジェネシスブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "000000052f976ab1744140530cc83a328ad2104790f7d1ab416ab970a6d401ad",
  "elapsed_time": "64.439秒",
  "index": 0,
  "nonce": "00f3f0a8",
  "prev_hash": "0000000000000000000000000000000000000000000000000000000000000000",
  "stored_data": "ジェネシスブロック",
  "timestamp": "2019/10/29 13:19:36"
}
2番目のブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "000000620b49d37cda8e14d0af3b05f7445ad9fcb911c9842b32710484e550a6",
  "elapsed_time": "343.687秒",
  "index": 1,
  "nonce": "0522b534",
  "prev_hash": "000000052f976ab1744140530cc83a328ad2104790f7d1ab416ab970a6d401ad",
  "stored_data": "ブロック 1",
  "timestamp": "2019/10/29 13:20:40"
}
3番目のブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "0000004c2ba01aa606d8fa08a604920ff806cd08bf4990d5f9d595d9084fbf71",
  "elapsed_time": "15.742秒",
  "index": 2,
  "nonce": "003ef9dd",
  "prev_hash": "000000620b49d37cda8e14d0af3b05f7445ad9fcb911c9842b32710484e550a6",
  "stored_data": "ブロック 2",
  "timestamp": "2

{
  "bits": "1d777777",
  "block_hash": "000000286f1584f4efd6d80bab9964617e3c5565b7e831fea76270702f9e5569",
  "elapsed_time": "211.65秒",
  "index": 24,
  "nonce": "02f1f08f",
  "prev_hash": "00000014759e397887f3f2fc18eeffe885a8a1a53956681a63f75888e7de7c6f",
  "stored_data": "ブロック 24",
  "timestamp": "2019/10/29 14:07:32"
}
26番目のブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "0000001e51087afbd655568ccca41c58a7ebfa361635cbdd01f6af37eb66f564",
  "elapsed_time": "173.358秒",
  "index": 25,
  "nonce": "0294d069",
  "prev_hash": "000000286f1584f4efd6d80bab9964617e3c5565b7e831fea76270702f9e5569",
  "stored_data": "ブロック 25",
  "timestamp": "2019/10/29 14:11:04"
}
27番目のブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "00000035d910c928556dbf7f737625f91c612f282efbb9e011a683d846255569",
  "elapsed_time": "389.875秒",
  "index": 26,
  "nonce": "0573037b",
  "prev_hash": "0000001e51087afbd655568ccca41c58a7ebfa361635cbdd01f6af37eb66f564",
  "stored_data": "ブロック 26",
  "timestamp": "2019/10/29 1